## This notebook show how to save memory and time

In [ ]:
from memory_profiler import profile

# instantiating the decorator
@profile
# code for which memory has to
# be monitored
def my_func():
	x = [x for x in range(0, 1000)]
	y = [y*100 for y in range(0, 1500)]
	del x
	return y

if __name__ == '__main__':
    my_func()

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd

In [10]:
exp_text = [(
    "Hold at 4.2V until C/100", 
    "Discharge at 1C until 2.5V", 
    "Rest for 1 hours", 
    "Charge at 1C until 4.2V", 
)]
param = pybamm.ParameterValues("OKane2022")
param.update({"Inner SEI lithium interstitial diffusivity [m2.s-1]": 3.5e-19})  


In [11]:
def Run_model_1st(exp_text,param,save_at_cycles,var_pts,):
    
    model = pybamm.lithium_ion.DFN(options={
        "SEI": "interstitial-diffusion limited", 
        "SEI film resistance": "distributed", 
        "SEI porosity change": "true",})
    exp = pybamm.Experiment( exp_text*save_at_cycles )
    sim = pybamm.Simulation(
        model, 
        experiment=exp,
        parameter_values=param,
        solver=pybamm.CasadiSolver(return_solution_if_failed_early=True),
        var_pts=var_pts,
    )
    sol = sim.solve(save_at_cycles=save_at_cycles,calc_esoh=False)
    
    return sol,model

def Run_model_later(model_old, sol_old, exp_text,param, save_at_cycles,var_pts,):
    model = model_old.set_initial_conditions_from(sol_old, inplace=False)
    exp = pybamm.Experiment( exp_text*save_at_cycles )
    sim = pybamm.Simulation(
        model, 
        experiment=exp,
        parameter_values=param,
        solver=pybamm.CasadiSolver(return_solution_if_failed_early=True),
        var_pts=var_pts,
    )
    sol = sim.solve(save_at_cycles=save_at_cycles,calc_esoh=False)
    return sol,model

def Post_processing(sol,cyc,step):
    time = (
        sol.cycles[cyc].steps[step]["Time [h]"].entries 
        -
        sol.cycles[cyc].steps[step]["Time [h]"].entries[0] )
    v = sol.cycles[cyc].steps[step]["Terminal voltage [V]"].entries
    return time, v

def Run_Big_Model(exp_text,param,save_at_cycles,var_pts,total_cycles,return_sol):
    start_1 = pybamm.Timer()
    Time = []; Voltage = []; Cyc_count = []; Sol_all = []
    i=1; n = total_cycles / save_at_cycles
    # Run the first N cycles:
    sol,model = Run_model_1st(exp_text,param,save_at_cycles,var_pts,)
    if return_sol == True:
        Sol_all.append(sol)
    time, v = Post_processing(sol,0,1)
    Cyc_count.append(i+1)
    Time.append(time)
    Voltage.append(v)
    time, v = Post_processing(sol,-1,1)
    Cyc_count.append(save_at_cycles)
    Time.append(time)
    Voltage.append(v)
    while i<n:
        sol,model = Run_model_later(model, sol, exp_text,param, save_at_cycles,var_pts,)
        if return_sol == True:
            Sol_all.append(sol)
        time, v = Post_processing(sol,-1,1)
        Time.append(time)
        Voltage.append(v)
        Cyc_count.append(save_at_cycles*i)
        i += 1
    print(f'running time is {start_1.time()}')
    return Time, Voltage, Cyc_count,Sol_all

var_pts={"x_n": 5, "x_s": 5,"x_p": 5, "r_n": 30, "r_p": 20, }
save_at_cycles = 3; total_cycles = 9


running time is 17.127 s


In [ ]:
Time_2, Voltage_2, Cyc_count_2,Sol_all_2 = Run_Big_Model(
    exp_text,param,save_at_cycles,var_pts,total_cycles,return_sol=False)

In [ ]:
#### record-1:
var_pts={"x_n": 5, "x_s": 5,"x_p": 5, "r_n": 30, "r_p": 20, }
save_at_cycles = 3; total_cycles = 9

running time is 56.980 s
Filename: /home/ruihe_li/EnvPB_Linux/PyBaMM/scripts/Ruihe_HPC/Strategies/test_ram.py

Line     Mem usage    Increment  Occurrences   Line Contents
=============================================================
    90    195.3 MiB    195.3 MiB           1   @profile
    91                                         def Main():
    92    299.3 MiB    104.1 MiB           2       Time, Voltage, Cyc_count,Sol_all = Run_Big_Model(
    93    195.3 MiB      0.0 MiB           1           exp_text,param,save_at_cycles,var_pts,
    94    195.3 MiB      0.0 MiB           1           total_cycles,return_sol=False)

#### record-2:
var_pts={"x_n": 5, "x_s": 5,"x_p": 5, "r_n": 30, "r_p": 20, }
save_at_cycles = 3; total_cycles = 9

running time is 56.560 s
Filename: /home/ruihe_li/EnvPB_Linux/PyBaMM/scripts/Ruihe_HPC/Strategies/test_ram.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
    90    201.4 MiB    201.4 MiB           1   @profile
    91                                         def Main():
    92    307.7 MiB    106.3 MiB           2       Time, Voltage, Cyc_count,Sol_all = Run_Big_Model(
    93    201.4 MiB      0.0 MiB           1           exp_text,param,save_at_cycles,var_pts,
    94    201.4 MiB      0.0 MiB           1           total_cycles,return_sol=True)

#### record-3:
var_pts={"x_n": 5, "x_s": 5,"x_p": 5, "r_n": 30, "r_p": 20, }
save_at_cycles = 100; total_cycles = 1000

running time is 18 minutes, 38 seconds
Filename: /home/ruihe_li/EnvPB_Linux/PyBaMM/scripts/Ruihe_HPC/Strategies/test_ram.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
    90    203.5 MiB    203.5 MiB           1   @profile
    91                                         def Main():
    92    363.5 MiB    160.0 MiB           2       Time, Voltage, Cyc_count,Sol_all = Run_Big_Model(
    93    203.5 MiB      0.0 MiB           1           exp_text,param,save_at_cycles,var_pts,
    94    203.5 MiB      0.0 MiB           1           total_cycles,return_sol=False)

#### record-4:

var_pts={"x_n": 5, "x_s": 5,"x_p": 5, "r_n": 30, "r_p": 20, }
save_at_cycles = 100; total_cycles = 1000

running time is 17 minutes, 43 seconds
Filename: /home/ruihe_li/EnvPB_Linux/PyBaMM/scripts/Ruihe_HPC/Strategies/test_ram.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
    90    201.3 MiB    201.3 MiB           1   @profile
    91                                         def Main():
    92    625.8 MiB    424.6 MiB           2       Time, Voltage, Cyc_count,Sol_all = Run_Big_Model(
    93    201.3 MiB      0.0 MiB           1           exp_text,param,save_at_cycles,var_pts,
    94    201.3 MiB      0.0 MiB           1           total_cycles,return_sol=True)

#### record-5:

var_pts={"x_n": 5, "x_s": 5,"x_p": 5, "r_n": 30, "r_p": 20, }
save_at_cycles = 5; total_cycles = 1000

running time is 1 hour, 1 minute, 1 second
Filename: /home/ruihe_li/EnvPB_Linux/PyBaMM/scripts/Ruihe_HPC/Strategies/test_ram.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
    90    203.6 MiB    203.6 MiB           1   @profile
    91                                         def Main():
    92    324.7 MiB    121.1 MiB           2       Time, Voltage, Cyc_count,Sol_all = Run_Big_Model(
    93    203.6 MiB      0.0 MiB           1           exp_text,param,save_at_cycles,var_pts,
    94    203.6 MiB      0.0 MiB           1           total_cycles,return_sol=False)

#### record-6:

var_pts={"x_n": 5, "x_s": 5,"x_p": 5, "r_n": 30, "r_p": 20, }
save_at_cycles = 5; total_cycles = 1000

running time is 56 minutes, 23 seconds
Filename: /home/ruihe_li/EnvPB_Linux/PyBaMM/scripts/Ruihe_HPC/Strategies/test_ram.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
    90    193.4 MiB    193.4 MiB           1   @profile
    91                                         def Main():
    92   4887.2 MiB   4693.9 MiB           2       Time, Voltage, Cyc_count,Sol_all = Run_Big_Model(
    93    193.4 MiB      0.0 MiB           1           exp_text,param,save_at_cycles,var_pts,
    94    193.4 MiB      0.0 MiB           1           total_cycles,return_sol=True)
